# Compressor Bottleneck Analysis with NeqSim

This notebook demonstrates process simulation with NeqSim, including:
- Building a multi-train gas processing system
- Configuring compressors with performance charts and drivers
- Analyzing pipe velocities and equipment capacity utilization
- Detecting system bottlenecks

The simulation replicates the `testBottleneck2` test case from the Java codebase.

In [28]:
year = 2026

## 1. Import Required Libraries and Setup NeqSim

Import NeqSim classes using direct Java access via jpype. This includes ProcessSystem, Stream, Splitter, Compressor, Separator, PipeBeggsAndBrills, Manifold, CompressorDriver, and StreamSaturatorUtil.

In [29]:
import neqsim
from neqsim import jneqsim

# Import Java classes for direct access
SystemPrEos = jneqsim.thermo.system.SystemPrEos
ProcessSystem = jneqsim.process.processmodel.ProcessSystem
Stream = jneqsim.process.equipment.stream.Stream
Splitter = jneqsim.process.equipment.splitter.Splitter
Compressor = jneqsim.process.equipment.compressor.Compressor
CompressorDriver = jneqsim.process.equipment.compressor.CompressorDriver
DriverType = jneqsim.process.equipment.compressor.DriverType
Separator = jneqsim.process.equipment.separator.Separator
ThreePhaseSeparator = jneqsim.process.equipment.separator.ThreePhaseSeparator
PipeBeggsAndBrills = jneqsim.process.equipment.pipeline.PipeBeggsAndBrills
Manifold = jneqsim.process.equipment.manifold.Manifold
StreamSaturatorUtil = jneqsim.process.equipment.util.StreamSaturatorUtil


## 2. Create Test Fluid System

Create a SystemPrEos fluid with natural gas composition using the Peng-Robinson equation of state.

In [30]:
# Create test fluid with natural gas composition
test_system = SystemPrEos(298.15, 10.0)  # Temperature in K, Pressure in bara
test_system.addComponent("nitrogen", 0.0015)
test_system.addComponent("methane", 0.925)
test_system.addComponent("ethane", 0.03563)
test_system.addComponent("propane", 0.00693)
test_system.addComponent("water", 0.001)
test_system.setMixingRule("classic")
test_system.setMultiPhaseCheck(True)


## 3. Build Inlet Stream and Water Saturator

Create the inlet stream with specified flow rate, temperature, and pressure. Add a water saturator to saturate the gas stream with water vapor.

In [31]:
# Create process system
process_system = ProcessSystem()

# Create inlet stream
inlet_stream = Stream("Inlet Stream", test_system)
inlet_stream.setFlowRate(2097870.58288790, "kg/hr")
inlet_stream.setTemperature(48.5, "C")
inlet_stream.setPressure(37.16, "bara")
inlet_stream.run()
process_system.add(inlet_stream)

# Saturate the stream with water
saturator = StreamSaturatorUtil("Water Saturator", inlet_stream)
saturator.run()
process_system.add(saturator)

# Get the water-saturated outlet stream
saturated_stream = Stream("Saturated Stream", saturator.getOutletStream())
saturated_stream.run()
process_system.add(saturated_stream)

print(f"Inlet Stream:")
print(f"  Flow rate: {inlet_stream.getFlowRate('kg/hr'):.0f} kg/hr")
print(f"  Temperature: {inlet_stream.getTemperature('C'):.1f} °C")
print(f"  Pressure: {inlet_stream.getPressure('bara'):.2f} bara")

Inlet Stream:
  Flow rate: 2097871 kg/hr
  Temperature: 48.5 °C
  Pressure: 37.16 bara


## 4. Create Splitter and Processing Trains

Create a 4-way splitter and processing trains. Each train consists of an inlet pipe, three-phase separator, and outlet pipe.

In [32]:
def create_processing_train(train_name, inlet_stream, process_system):
    """Creates a processing train with pipe, separator, and outlet pipe."""
    
    # Inlet pipe
    inlet_pipe = PipeBeggsAndBrills(train_name + " Inlet Pipe", inlet_stream)
    inlet_pipe.setLength(100.0)  # meters
    inlet_pipe.setDiameter(0.7)  # meters
    inlet_pipe.setPipeWallRoughness(15e-6)
    inlet_pipe.setElevation(0)
    inlet_pipe.run()
    process_system.add(inlet_pipe)
    
    # Three-phase separator
    separator = ThreePhaseSeparator(train_name + " Separator", inlet_pipe.getOutletStream())
    separator.run()
    process_system.add(separator)
    
    # Outlet pipe
    outlet_pipe = PipeBeggsAndBrills(train_name + " Outlet Pipe", separator.getGasOutStream())
    outlet_pipe.setLength(100.0)  # meters
    outlet_pipe.setDiameter(0.7)  # meters
    outlet_pipe.setPipeWallRoughness(15e-6)
    outlet_pipe.setElevation(0)
    outlet_pipe.setNumberOfIncrements(10)
    outlet_pipe.run()
    process_system.add(outlet_pipe)
    
    return outlet_pipe

# Create first splitter
splitter = Splitter("Test Splitter", saturated_stream)
splitter.setSplitFactors([0.25, 0.25, 0.25, 0.25])
splitter.run()
process_system.add(splitter)

# Create processing trains for each split stream
train1_outlet = create_processing_train("Train1", splitter.getSplitStream(0), process_system)
train2_outlet = create_processing_train("Train2", splitter.getSplitStream(1), process_system)
train3_outlet = create_processing_train("Train3", splitter.getSplitStream(2), process_system)
train4_outlet = create_processing_train("Train4", splitter.getSplitStream(3), process_system)

print("Created 4 processing trains with pipes and separators")

Created 4 processing trains with pipes and separators


## 5. Build Final Separator and Second Splitter

Combine all four processing train outlets in a final separator, then split to three compressor trains.

In [33]:
# Create final separator combining all train outlets
final_separator = ThreePhaseSeparator("Final Separator")
final_separator.addStream(train1_outlet.getOutletStream())
final_separator.addStream(train2_outlet.getOutletStream())
final_separator.addStream(train3_outlet.getOutletStream())
final_separator.addStream(train4_outlet.getOutletStream())
final_separator.setInternalDiameter(3.0)
final_separator.run()
process_system.add(final_separator)

# Create second splitter for compressor trains (slightly unequal distribution)
splitter2 = Splitter("Test Splitter2", final_separator.getGasOutStream())
splitter2.setSplitFactors([0.95 / 3.0, 1.0 / 3.0, 1.05 / 3.0])
splitter2.run()
process_system.add(splitter2)

print(f"Final separator internal diameter: {final_separator.getInternalDiameter()} m")
print(f"Splitter 2 split factors: {[0.95/3.0, 1.0/3.0, 1.05/3.0]}")

Final separator internal diameter: 3.0 m
Splitter 2 split factors: [0.31666666666666665, 0.3333333333333333, 0.35000000000000003]


## 6. Create Upstream Compressor Trains

Create three compressor trains, each with inlet pipe, separator, outlet pipe, and compressor.

In [34]:
def create_upstream_compressors(train_name, inlet_stream, process_system):
    """Creates an upstream compressor train with pipes, separator, and compressor."""
    
    # Print inlet conditions
    print(f"\n=== {train_name} INLET CONDITIONS ===")
    print(f"Inlet Pressure: {inlet_stream.getPressure('bara'):.2f} bara")
    print(f"Inlet Flow: {inlet_stream.getFlowRate('kg/hr'):.0f} kg/hr")
    print(f"Gas Density: {inlet_stream.getFluid().getDensity('kg/m3'):.2f} kg/m3")
    
    # Inlet pipe
    inlet_pipe = PipeBeggsAndBrills(train_name + " ups Pipe", inlet_stream)
    inlet_pipe.setLength(100)  # meters
    inlet_pipe.setDiameter(0.75)  # meters
    inlet_pipe.setPipeWallRoughness(15e-6)
    inlet_pipe.setElevation(0)
    inlet_pipe.run()
    process_system.add(inlet_pipe)
    
    # Separator
    separator = Separator(train_name + " ups Separator", inlet_pipe.getOutletStream())
    separator.run()
    process_system.add(separator)
    
    # Outlet pipe
    outlet_pipe = PipeBeggsAndBrills(train_name + " ups Outlet Pipe", separator.getGasOutStream())
    outlet_pipe.setLength(50.0)  # meters
    outlet_pipe.setDiameter(0.75)  # meters
    outlet_pipe.setPipeWallRoughness(15e-6)
    outlet_pipe.setElevation(0)
    outlet_pipe.run()
    process_system.add(outlet_pipe)
    
    # Compressor
    compressor = Compressor(train_name + " Compressor", outlet_pipe.getOutletStream())
    compressor.setOutletPressure(110.0, "bara")
    compressor.setUsePolytropicCalc(True)
    compressor.setPolytropicEfficiency(0.85)
    compressor.setSpeed(8000)
    compressor.run()
    process_system.add(compressor)

    # Outlet pipe
    outlet_pipe2 = PipeBeggsAndBrills(train_name + " ups Outlet Pipe2", compressor.getOutletStream())
    outlet_pipe2.setLength(50.0)  # meters
    outlet_pipe2.setDiameter(0.75)  # meters
    outlet_pipe2.setPipeWallRoughness(15e-6)
    outlet_pipe2.setElevation(0)
    outlet_pipe2.run()
    process_system.add(outlet_pipe2)
    
    return outlet_pipe2.getOutletStream()

# Create upstream compressor trains
ups1_outlet = create_upstream_compressors("ups1", splitter2.getSplitStream(0), process_system)
ups2_outlet = create_upstream_compressors("ups2", splitter2.getSplitStream(1), process_system)
ups3_outlet = create_upstream_compressors("ups3", splitter2.getSplitStream(2), process_system)


=== ups1 INLET CONDITIONS ===
Inlet Pressure: 37.08 bara
Inlet Flow: 665658 kg/hr
Gas Density: 24.68 kg/m3

=== ups2 INLET CONDITIONS ===
Inlet Pressure: 37.08 bara
Inlet Flow: 700692 kg/hr
Gas Density: 24.68 kg/m3

=== ups3 INLET CONDITIONS ===
Inlet Pressure: 37.08 bara
Inlet Flow: 735727 kg/hr
Gas Density: 24.68 kg/m3


## 7. Create Manifold and Run Process System

Create a manifold to combine all three compressor outlets and run the complete process system.

In [35]:
# Create manifold combining compressor outlets
manifold = Manifold("Compressor Outlet Manifold")
manifold.addStream(ups1_outlet)
manifold.addStream(ups2_outlet)
manifold.addStream(ups3_outlet)
manifold.setSplitFactors([1.0 / 3.0, 1.0 / 3.0, 1.0 / 3.0])
manifold.setHeaderInnerDiameter(1.5)
manifold.setBranchInnerDiameter(0.6)
manifold.run()
process_system.add(manifold)

# Run the process system
process_system.run()

print("\nProcess system created and run successfully!")
print(f"Total unit operations: {process_system.getUnitOperations().size()}")


Process system created and run successfully!
Total unit operations: 34


## 8. Auto-size Equipment

Auto-size separators, compressors, and manifolds based on current operating conditions.

In [36]:
# Auto-size separators and compressors
for equipment in process_system.getUnitOperations():
    if isinstance(equipment, Separator):
        equipment.autoSize()
    elif isinstance(equipment, Compressor):
        equipment.autoSize()
    elif isinstance(equipment, Manifold):
        equipment.autoSize()

print("Equipment auto-sizing complete")

Equipment auto-sizing complete


## 9. Configure Compressor Charts and Drivers

Load compressor performance charts from JSON files and configure gas turbine drivers with speed-dependent power curves.

In [37]:
# Configure ups1 compressor with driver curve
ups1_comp = process_system.getUnit("ups1 Compressor")
ups1_comp.getMechanicalDesign().setMaxDesignPower(50000.0)
ups1_comp.loadCompressorChartFromJson("C:/Users/ESOL/Documents/GitHub/neqsim2/src/test/resources/compressor_curves/example_compressor_curve.json")
ups1_comp.setSolveSpeed(True)

# Set up driver with speed-dependent max power curve
# P_max(N) = maxPower * (a + b*(N/N_rated) + c*(N/N_rated)^2)
driver1 = CompressorDriver(DriverType.GAS_TURBINE, 40500.0)
driver1.setRatedSpeed(7383.0)
driver1.setMaxPowerCurveCoefficients(0.3, 0.5, 0.2)  # ~0.86 at 70% speed, 1.0 at 100%
ups1_comp.setDriver(driver1)

# Configure ups2 compressor
ups2_comp = process_system.getUnit("ups2 Compressor")
ups2_comp.getMechanicalDesign().setMaxDesignPower(50000.0)
ups2_comp.loadCompressorChartFromJson("C:/Users/ESOL/Documents/GitHub/neqsim2/src/test/resources/compressor_curves/compressor_curve_ups2.json")
ups2_comp.setSolveSpeed(True)
driver2 = CompressorDriver(DriverType.GAS_TURBINE, 40500.0)
driver2.setRatedSpeed(7383.0)
driver2.setMaxPowerCurveCoefficients(0.3, 0.5, 0.2)
ups2_comp.setDriver(driver2)

# Configure ups3 compressor (different driver specs)
ups3_comp = process_system.getUnit("ups3 Compressor")
ups3_comp.getMechanicalDesign().setMaxDesignPower(50000.0)
ups3_comp.loadCompressorChartFromJson("C:/Users/ESOL/Documents/GitHub/neqsim2/src/test/resources/compressor_curves/compressor_curve_ups3.json")
ups3_comp.setSolveSpeed(True)
driver3 = CompressorDriver(DriverType.GAS_TURBINE, 45000.0)
driver3.setRatedSpeed(6726.3)  # Different rated speed for ups3
driver3.setMaxPowerCurveCoefficients(0.3, 0.5, 0.2)
ups3_comp.setDriver(driver3)

# Run system with configured compressors
process_system.run()

print("Compressor charts and drivers configured:")
print(f"  ups1: Rated Power = {driver1.getRatedPower():.0f} kW, Rated Speed = {driver1.getRatedSpeed():.0f} RPM")
print(f"  ups2: Rated Power = {driver2.getRatedPower():.0f} kW, Rated Speed = {driver2.getRatedSpeed():.0f} RPM")
print(f"  ups3: Rated Power = {driver3.getRatedPower():.0f} kW, Rated Speed = {driver3.getRatedSpeed():.0f} RPM")

Compressor charts and drivers configured:
  ups1: Rated Power = 40500 kW, Rated Speed = 7383 RPM
  ups2: Rated Power = 40500 kW, Rated Speed = 7383 RPM
  ups3: Rated Power = 45000 kW, Rated Speed = 6726 RPM


## 10. Run Flow Propagation Test

Test that flow changes propagate correctly through the system by changing the inlet flow rate.

In [38]:
# Get initial compressor flow
initial_flow = process_system.getUnit("ups1 Compressor").getInletStream().getFlowRate("kg/hr")

# Change inlet flow rate and run
#inlet_stream.setFlowRate(2157870.58288790, "kg/hr")
#process_system.run()

# Verify flow propagation
new_flow = process_system.getUnit("ups1 Compressor").getInletStream().getFlowRate("kg/hr")

print("=== FLOW PROPAGATION TEST ===")
print(f"Initial flow: {initial_flow:.2f} kg/hr")
print(f"New flow: {new_flow:.2f} kg/hr")
print(f"Flow change: {abs(new_flow - initial_flow):.2f} kg/hr")

=== FLOW PROPAGATION TEST ===
Initial flow: 665657.70 kg/hr
New flow: 665657.70 kg/hr
Flow change: 0.00 kg/hr


## 11. Analyze Pipe Velocities

Analyze superficial velocities in all pipes and compare against the maximum allowed velocity (30 m/s for gas pipes).

In [39]:
print("=== PIPE VELOCITY ANALYSIS ===")
max_allowed_velocity = 30.0  # m/s - max for gas pipes

# Print header
print(f"{'Pipe Name':<25}  {'Vin':>8}  {'Vout':>8}  {'Vmax':>10}  {'Diameter':>10}  {'Flow':>10}  {'Status':>8}")
print(f"{'':<25}  {'(m/s)':>8}  {'(m/s)':>8}  {'(m/s)':>10}  {'(m)':>10}  {'(kg/hr)':>10}  {''}")
print("-" * 100)

# Analyze each pipe (use class name check for Java objects)
for equipment in process_system.getUnitOperations():
    class_name = equipment.getClass().getSimpleName()
    if class_name == "PipeBeggsAndBrills":
        pipe = equipment
        v_in = float(pipe.getInletSuperficialVelocity())
        v_out = float(pipe.getOutletSuperficialVelocity())
        v_max = max(v_in, v_out)
        diameter = float(pipe.getDiameter())
        flow_rate = float(pipe.getInletStream().getFlowRate("kg/hr"))
        status = "HIGH!" if v_max > max_allowed_velocity else "OK"
        utilization = (v_max / max_allowed_velocity) * 100.0
        print('pipe.getName()',  v_in)

=== PIPE VELOCITY ANALYSIS ===
Pipe Name                       Vin      Vout        Vmax    Diameter        Flow    Status
                              (m/s)     (m/s)       (m/s)         (m)     (kg/hr)  
----------------------------------------------------------------------------------------------------
pipe.getName() 15.332227602531919
pipe.getName() 15.34923444872278
pipe.getName() 15.332227602531919
pipe.getName() 15.34923444872278
pipe.getName() 15.332227602531919
pipe.getName() 15.34923444872278
pipe.getName() 15.332227602531919
pipe.getName() 15.34923444872278
pipe.getName() 16.95528617161652
pipe.getName() 16.976456313954657
pipe.getName() 7.786258298601355
pipe.getName() 17.847669654333178
pipe.getName() 17.872335634110613
pipe.getName() 8.214612870866484
pipe.getName() 18.74005313704984
pipe.getName() 18.768580585653684
pipe.getName() 8.581741024293125


## 12. Display Equipment Capacity Utilization

Get the capacity utilization summary for all equipment in the process system.

In [40]:
print("=== EQUIPMENT CAPACITY UTILIZATION ===")
utilization_summary = process_system.getCapacityUtilizationSummary()

for entry in utilization_summary.entrySet():
    print(entry.getKey(), entry.getValue())

=== EQUIPMENT CAPACITY UTILIZATION ===
Train1 Inlet Pipe 76.74617224361391
Train1 Separator 81.68050502274912
Train1 Outlet Pipe 76.83148607573283
Train2 Inlet Pipe 76.74617224361391
Train2 Separator 81.68050502274912
Train2 Outlet Pipe 76.83148607573283
Train3 Inlet Pipe 76.74617224361391
Train3 Separator 81.68050502274912
Train3 Outlet Pipe 76.83148607573283
Train4 Inlet Pipe 76.74617224361391
Train4 Separator 81.68050502274912
Train4 Outlet Pipe 76.83148607573283
Final Separator 81.67981142756156
ups1 ups Pipe 84.88228156977328
ups1 ups Separator 81.67903336904968
ups1 ups Outlet Pipe 84.93535457072558
ups1 Compressor 97.27458869186836
ups1 ups Outlet Pipe2 38.93486356347402
ups2 ups Pipe 89.36167817055306
ups2 ups Separator 81.67895033043843
ups2 ups Outlet Pipe 89.42353356373219
ups2 Compressor 98.61009720570037
ups2 ups Outlet Pipe2 41.077242131125345
ups3 ups Pipe 93.84290292826843
ups3 ups Separator 81.67886308308925
ups3 ups Outlet Pipe 93.91446420174843
ups3 Compressor 95.383

## 13. Analyze Compressor Power and Margins

Detailed analysis of the ups1 compressor including power utilization, speed, head, efficiency, and operating margins.

In [41]:
comp = process_system.getUnit("ups1 Compressor")

if comp is not None:
    power_kw = comp.getPower("kW")
    max_power = comp.getMechanicalDesign().maxDesignPower
    
    print("=== COMPRESSOR POWER ===")
    print(f"Compressor: {comp.getName()}")
    print(f"Power: {power_kw:.2f} kW ({power_kw/1000:.2f} MW)")
    if max_power > 0:
        print(f"Max Design Power: {max_power:.2f} kW ({max_power/1000:.2f} MW)")
        print(f"Power Utilization: {(power_kw / max_power) * 100:.2f}%")
    
    print("\n=== COMPRESSOR MARGINS ===")
    print(f"Speed: {comp.getSpeed():.0f} RPM")
    print(f"Max Speed: {comp.getMaximumSpeed():.0f} RPM")
    print(f"Min Speed: {comp.getMinimumSpeed():.0f} RPM")
    print(f"Chart Max Speed: {comp.getCompressorChart().getMaxSpeedCurve():.0f} RPM")
    print(f"Chart Min Speed: {comp.getCompressorChart().getMinSpeedCurve():.0f} RPM")
    print(f"Polytropic Head: {comp.getPolytropicFluidHead():.0f} J/kg")
    print(f"Polytropic Efficiency: {comp.getPolytropicEfficiency() * 100:.1f}%")
    print(f"Distance to Surge: {comp.getDistanceToSurge() * 100:.2f}%")
    print(f"Distance to Stonewall: {comp.getDistanceToStoneWall() * 100:.2f}%")
    print(f"Inlet Volume Flow: {comp.getInletStream().getFlowRate('m3/hr'):.0f} m3/hr")
    print(f"Inlet Pressure: {comp.getInletStream().getPressure('bara'):.2f} bara")
    print(f"Outlet Pressure: {comp.getOutletStream().getPressure('bara'):.2f} bara")

=== COMPRESSOR POWER ===
Compressor: ups1 Compressor
Power: 40679.98 kW (40.68 MW)
Max Design Power: 50000.00 kW (50.00 MW)
Power Utilization: 81.36%

=== COMPRESSOR MARGINS ===
Speed: 6872 RPM
Max Speed: 7383 RPM
Min Speed: 4922 RPM
Chart Max Speed: 7383 RPM
Chart Min Speed: 4922 RPM
Polytropic Head: 185 J/kg
Polytropic Efficiency: 84.3%
Distance to Surge: 75.15%
Distance to Stonewall: 146.48%
Inlet Volume Flow: 27017 m3/hr
Inlet Pressure: 37.02 bara
Outlet Pressure: 110.00 bara


## 14. Display Driver Curve Information

Show driver information for all compressors including speed-dependent maximum power at current operating speed.

In [42]:
print("=== DRIVER CURVE (Speed-Dependent Max Power) ===")

for equipment in process_system.getUnitOperations():
    if isinstance(equipment, Compressor):
        compressor = equipment
        driver = compressor.getDriver()
        if driver is not None:
            print(f"\nCompressor: {compressor.getName()}")
            print(f"  Driver Type: {driver.getDriverType()}")
            print(f"  Rated Power: {driver.getRatedPower():.0f} kW")
            print(f"  Rated Speed: {driver.getRatedSpeed():.0f} RPM")
            print(f"  Current Speed: {compressor.getSpeed():.0f} RPM")
            max_power_at_speed = driver.getMaxAvailablePowerAtSpeed(compressor.getSpeed())
            print(f"  Max Power at Current Speed: {max_power_at_speed:.0f} kW")
            print(f"  Actual Power: {compressor.getPower('kW'):.0f} kW")
            power_utilization = compressor.getPower("kW") / max_power_at_speed * 100.0
            print(f"  Power Utilization (vs speed-dependent max): {power_utilization:.1f}%")

=== DRIVER CURVE (Speed-Dependent Max Power) ===

Compressor: ups1 Compressor
  Driver Type: GAS_TURBINE
  Rated Power: 40500 kW
  Rated Speed: 7383 RPM
  Current Speed: 6872 RPM
  Max Power at Current Speed: 41820 kW
  Actual Power: 40680 kW
  Power Utilization (vs speed-dependent max): 97.3%

Compressor: ups2 Compressor
  Driver Type: GAS_TURBINE
  Rated Power: 40500 kW
  Rated Speed: 7383 RPM
  Current Speed: 7256 RPM
  Max Power at Current Speed: 43862 kW
  Actual Power: 43252 kW
  Power Utilization (vs speed-dependent max): 98.6%

Compressor: ups3 Compressor
  Driver Type: GAS_TURBINE
  Rated Power: 45000 kW
  Rated Speed: 6726 RPM
  Current Speed: 6274 RPM
  Max Power at Current Speed: 46547 kW
  Actual Power: 44398 kW
  Power Utilization (vs speed-dependent max): 95.4%


## 15. Display Compressor Capacity Constraints

Show all capacity constraints for the ups1 compressor including surge margin, speed limits, and power utilization.

In [54]:
print("=== COMPRESSOR CAPACITY CONSTRAINTS ===")
comp = process_system.getUnit("ups1 Compressor")

constraints = comp.getCapacityConstraints()
for entry in constraints.entrySet():
    constraint = entry.getValue()
    label_type = "min" if constraint.isMinimumConstraint() else "design"
    print(entry.getKey(), constraint.getUtilizationPercent(),constraint.getCurrentValue(),constraint.getDisplayDesignValue())

=== COMPRESSOR CAPACITY CONSTRAINTS ===
speed 79.12287345173613 6329.8298761388905 8000.0
minSpeed 84.25713546327714 6329.8298761388905 5333.333333333333
power 44.78973984468155 44.78973984468155 100.0
surgeMargin 100.0 100.0 100.0
stonewallMargin 13.734120465562299 13.7341204655623 100.0


In [56]:
# Check all compressor constraints and overall capacity
print("=== ALL COMPRESSOR CAPACITY ANALYSIS ===")
for equipment in process_system.getUnitOperations():
    if isinstance(equipment, Compressor):
        comp = equipment
        print(f"\n{comp.getName()}:")
        constraints = comp.getCapacityConstraints()
        for entry in constraints.entrySet():
            constraint = entry.getValue()
            print(f"  {entry.getKey()}: util={constraint.getUtilizationPercent():.1f}%, current={constraint.getCurrentValue():.2f}, limit={constraint.getDisplayDesignValue():.2f}")

=== ALL COMPRESSOR CAPACITY ANALYSIS ===

ups1 Compressor:
  speed: util=79.1%, current=6329.83, limit=8000.00
  minSpeed: util=84.3%, current=6329.83, limit=5333.33
  power: util=44.8%, current=44.79, limit=100.00
  surgeMargin: util=100.0%, current=100.00, limit=100.00
  stonewallMargin: util=13.7%, current=13.73, limit=100.00

ups2 Compressor:
  speed: util=85.6%, current=6850.58, limit=8000.00
  minSpeed: util=77.9%, current=6850.58, limit=5333.33
  power: util=55.1%, current=55.11, limit=100.00
  surgeMargin: util=100.0%, current=100.00, limit=100.00
  stonewallMargin: util=32.5%, current=32.51, limit=100.00

ups3 Compressor:
  speed: util=73.1%, current=5849.36, limit=8000.00
  minSpeed: util=91.2%, current=5849.36, limit=5333.33
  power: util=63.6%, current=63.58, limit=100.00
  surgeMargin: util=91.6%, current=91.58, limit=100.00
  stonewallMargin: util=25.8%, current=25.79, limit=100.00


In [58]:
# Check pipe velocity constraints too
print("=== ALL PIPE CAPACITY ANALYSIS ===")
for equipment in process_system.getUnitOperations():
    class_name = equipment.getClass().getSimpleName()
    if class_name == "PipeBeggsAndBrills":
        pipe = equipment
        print(f"\n{pipe.getName()}:")
        max_v = max(float(pipe.getInletSuperficialVelocity()), float(pipe.getOutletSuperficialVelocity()))
        max_design_v = pipe.getMechanicalDesign().getMaxDesignVelocity()
        util = (max_v / max_design_v) * 100 if max_design_v > 0 else 0
        print(f"  Velocity: {max_v:.1f} m/s, Max Design: {max_design_v:.1f} m/s, Utilization: {util:.1f}%")

=== ALL PIPE CAPACITY ANALYSIS ===

Train1 Inlet Pipe:
  Velocity: 7.7 m/s, Max Design: 20.0 m/s, Utilization: 38.3%

Train1 Outlet Pipe:
  Velocity: 7.7 m/s, Max Design: 20.0 m/s, Utilization: 38.4%

Train2 Inlet Pipe:
  Velocity: 7.7 m/s, Max Design: 20.0 m/s, Utilization: 38.3%

Train2 Outlet Pipe:
  Velocity: 7.7 m/s, Max Design: 20.0 m/s, Utilization: 38.4%

Train3 Inlet Pipe:
  Velocity: 7.7 m/s, Max Design: 20.0 m/s, Utilization: 38.3%

Train3 Outlet Pipe:
  Velocity: 7.7 m/s, Max Design: 20.0 m/s, Utilization: 38.4%

Train4 Inlet Pipe:
  Velocity: 7.7 m/s, Max Design: 20.0 m/s, Utilization: 38.3%

Train4 Outlet Pipe:
  Velocity: 7.7 m/s, Max Design: 20.0 m/s, Utilization: 38.4%

ups1 ups Pipe:
  Velocity: 6.2 m/s, Max Design: 20.0 m/s, Utilization: 31.2%

ups1 ups Outlet Pipe:
  Velocity: 6.2 m/s, Max Design: 20.0 m/s, Utilization: 31.2%

ups1 ups Outlet Pipe2:
  Velocity: 3.0 m/s, Max Design: 20.0 m/s, Utilization: 14.9%

ups2 ups Pipe:
  Velocity: 8.9 m/s, Max Design: 20.0 m/

In [60]:
# Debug: Check raw compressor values that cause high utilization
print("=== DEBUG: RAW COMPRESSOR VALUES ===")
for equipment in process_system.getUnitOperations():
    if isinstance(equipment, Compressor):
        comp = equipment
        print(f"\n{comp.getName()}:")
        print(f"  Speed: {comp.getSpeed():.0f} RPM")
        print(f"  Inlet Flow: {comp.getInletStream().getFlowRate('m3/hr'):.0f} m3/hr")
        print(f"  Polytropic Head: {comp.getPolytropicFluidHead():.0f} J/kg")
        print(f"  Distance to Surge (raw): {comp.getDistanceToSurge():.6f}")
        print(f"  Distance to Stonewall (raw): {comp.getDistanceToStoneWall():.6f}")
        
        # If surge margin is very small or negative, it causes huge utilization!
        surge_margin = comp.getDistanceToSurge()
        if surge_margin > -1:  # Avoid division by zero
            surge_util = 100.0 / (1.0 + surge_margin)
            print(f"  Calculated Surge Utilization: {surge_util:.1f}%")
        else:
            print(f"  Calculated Surge Utilization: INVALID (margin <= -1)")
        
        # Check chart boundaries
        chart = comp.getCompressorChart()
        if chart is not None:
            print(f"  Chart Min Speed: {chart.getMinSpeedCurve():.0f} RPM")
            print(f"  Chart Max Speed: {chart.getMaxSpeedCurve():.0f} RPM")

=== DEBUG: RAW COMPRESSOR VALUES ===

ups1 Compressor:
  Speed: 6426 RPM
  Inlet Flow: 15894 m3/hr
  Polytropic Head: 186 J/kg
  Distance to Surge (raw): 0.029023
  Distance to Stonewall (raw): 3.201588
  Calculated Surge Utilization: 97.2%
  Chart Min Speed: 4922 RPM
  Chart Max Speed: 7383 RPM

ups2 Compressor:
  Speed: 7054 RPM
  Inlet Flow: 22721 m3/hr
  Polytropic Head: 185 J/kg
  Distance to Surge (raw): 0.241192
  Distance to Stonewall (raw): 0.910572
  Calculated Surge Utilization: 80.6%
  Chart Min Speed: 4922 RPM
  Chart Max Speed: 7383 RPM

ups3 Compressor:
  Speed: 905 RPM
  Inlet Flow: 29564 m3/hr
  Polytropic Head: 0 J/kg
  Distance to Surge (raw): 7.115628
  Distance to Stonewall (raw): -0.926044
  Calculated Surge Utilization: 12.3%
  Chart Min Speed: 4484 RPM
  Chart Max Speed: 6726 RPM


## 16. Detect Bottleneck Equipment

Identify equipment near capacity limits and detect the system bottleneck.

In [57]:
# Equipment near capacity limit (>90%)
print("=== EQUIPMENT NEAR CAPACITY LIMIT (>90%) ===")
near_limit = process_system.getEquipmentNearCapacityLimit()
if near_limit.isEmpty():
    print("No equipment near capacity limit")
else:
    for name in near_limit:
        print(f"  - {name}")

# Bottleneck detection
bottleneck = process_system.findBottleneck()
print("\n=== BOTTLENECK ANALYSIS ===")
if bottleneck is not None and bottleneck.hasBottleneck():
    print(f"Bottleneck Equipment: {bottleneck.getEquipmentName()}")
    print(f"Constraint: {bottleneck.getConstraintName()}")
    print(f"Utilization: {bottleneck.getUtilizationPercent():.2f}%")
else:
    print("No bottleneck detected")

=== EQUIPMENT NEAR CAPACITY LIMIT (>90%) ===
  - ups1 Compressor
  - ups2 Compressor
  - ups3 Compressor

=== BOTTLENECK ANALYSIS ===
Bottleneck Equipment: ups1 Compressor
Constraint: surgeMargin
Utilization: 100.00%


## 17. Production Optimization

This section demonstrates production optimization using NeqSim's `ProductionOptimizer` including:
- Single-variable optimization (flow rate)
- Multi-variable optimization (flow + split factors)
- Pareto multi-objective optimization (throughput vs power)

In [45]:
from jpype import JImplements, JOverride
import jpype

# Import ProductionOptimizer and inner classes
ProductionOptimizer = jneqsim.process.util.optimizer.ProductionOptimizer
OptimizationConfig = ProductionOptimizer.OptimizationConfig
OptimizationObjective = ProductionOptimizer.OptimizationObjective
OptimizationResult = ProductionOptimizer.OptimizationResult
ManipulatedVariable = ProductionOptimizer.ManipulatedVariable
ParetoResult = ProductionOptimizer.ParetoResult
SearchMode = ProductionOptimizer.SearchMode
ObjectiveType = ProductionOptimizer.ObjectiveType

# Import Java collections helpers
Collections = jpype.JClass("java.util.Collections")
Arrays = jpype.JClass("java.util.Arrays")
JDouble = jpype.JClass("java.lang.Double")

### 17.1 Single-Variable Optimization (Flow Rate)

Optimize the inlet flow rate to maximize throughput while respecting equipment capacity constraints.

In [59]:
# Reset inlet flow rate to original value and re-run system
inlet_stream.setFlowRate(2097870.58288790, "kg/hr")
process_system.run()

# Store original flow rate for comparison
original_flow = inlet_stream.getFlowRate("kg/hr")
current_flow = original_flow

print(f"Current inlet flow rate: {current_flow:,.0f} kg/hr")

# Initialize mechanical design for all pipes (required for velocity-based capacity checks)
print("\nInitializing mechanical design for pipes...")
for unit in process_system.getUnitOperations():
    if isinstance(unit, PipeBeggsAndBrills):
        unit.initMechanicalDesign()
        unit.getMechanicalDesign().setMaxDesignVelocity(20.0)  # Set max velocity 20 m/s
print("Mechanical design initialized for all pipes.")

# Create optimizer
optimizer = ProductionOptimizer()

# Configure optimization bounds and settings
# Use a smaller range to avoid extreme values that break compressor calculations
low_flow = current_flow * 0.8
high_flow = current_flow * 1.2

# Use BINARY_FEASIBILITY to find maximum feasible rate (respects 100% utilization limit)
single_var_config = OptimizationConfig(low_flow, high_flow) \
    .rateUnit("kg/hr") \
    .tolerance(current_flow * 0.005) \
    .maxIterations(25) \
    .defaultUtilizationLimit(1.0) \
    .searchMode(SearchMode.BINARY_FEASIBILITY)  # Finds max feasible rate

# Define throughput maximization objective
@JImplements("java.util.function.ToDoubleFunction")
class ThroughputEvaluator:
    @JOverride
    def applyAsDouble(self, proc):
        return proc.getUnit("Inlet Stream").getFlowRate("kg/hr")

throughput_objective = OptimizationObjective(
    "throughput", 
    ThroughputEvaluator(), 
    1.0,
    ObjectiveType.MAXIMIZE
)

# Run single-variable optimization
print(f"\nSearch range: {low_flow:,.0f} to {high_flow:,.0f} kg/hr")
print("Running single-variable optimization (BINARY_FEASIBILITY mode)...")
single_var_result = optimizer.optimize(
    process_system, 
    inlet_stream,
    single_var_config, 
    Collections.singletonList(throughput_objective), 
    Collections.emptyList()
)

# Print single-variable optimization results
print("=" * 50)
print("=== SINGLE-VARIABLE OPTIMIZATION RESULTS ===")
print("=" * 50)
print(f"Optimal flow rate: {single_var_result.getOptimalRate():,.0f} kg/hr")
print(f"Feasible: {single_var_result.isFeasible()}")
print(f"Iterations: {single_var_result.getIterations()}")

if single_var_result.getBottleneck() is not None:
    print(f"Bottleneck: {single_var_result.getBottleneck().getName()}")
    print(f"Bottleneck utilization: {single_var_result.getBottleneckUtilization() * 100.0:.1f}%")

# Print iteration history to understand the search
print("\n=== ITERATION HISTORY ===")
for i, rec in enumerate(single_var_result.getIterationHistory()):
    status = "FEASIBLE" if rec.isFeasible() else "INFEASIBLE"
    print(f"  Iter {i+1}: rate={rec.getRate():,.0f} kg/hr, util={rec.getBottleneckUtilization()*100:.1f}%, {status}")

Current inlet flow rate: 2,097,871 kg/hr

Initializing mechanical design for pipes...
Mechanical design initialized for all pipes.

Search range: 1,678,296 to 2,517,445 kg/hr
Running single-variable optimization (BINARY_FEASIBILITY mode)...
=== SINGLE-VARIABLE OPTIMIZATION RESULTS ===
Optimal flow rate: 1,678,296 kg/hr
Feasible: False
Iterations: 7
Bottleneck: ups2 Compressor
Bottleneck utilization: 69000.8%

=== ITERATION HISTORY ===
  Iter 1: rate=2,097,871 kg/hr, util=86504.3%, INFEASIBLE
  Iter 2: rate=1,888,084 kg/hr, util=77364.0%, INFEASIBLE
  Iter 3: rate=1,783,190 kg/hr, util=73054.9%, INFEASIBLE
  Iter 4: rate=1,730,743 kg/hr, util=70988.6%, INFEASIBLE
  Iter 5: rate=1,704,520 kg/hr, util=69985.4%, INFEASIBLE
  Iter 6: rate=1,691,408 kg/hr, util=69490.4%, INFEASIBLE
  Iter 7: rate=1,684,852 kg/hr, util=69490.4%, INFEASIBLE
  Iter 8: rate=1,678,296 kg/hr, util=69000.8%, INFEASIBLE


In [47]:
# Calculate production increase potential
best_feasible_flow = single_var_result.getOptimalRate()
production_increase = best_feasible_flow - original_flow
increase_percent = (production_increase / original_flow) * 100.0

print("=" * 50)
print("=== PRODUCTION POTENTIAL ===")
print("=" * 50)
print(f"Current production: {original_flow:,.0f} kg/hr")
print(f"Maximum production: {best_feasible_flow:,.0f} kg/hr")

if production_increase > 0:
    print(f"Potential increase: {production_increase:,.0f} kg/hr (+{increase_percent:.1f}%)")
elif production_increase < 0:
    print(f"Current production exceeds capacity - reduce by: {abs(production_increase):,.0f} kg/hr ({abs(increase_percent):.1f}%)")
else:
    print("Operating at optimal capacity")

=== PRODUCTION POTENTIAL ===
Current production: 2,097,871 kg/hr
Maximum production: 1,048,935 kg/hr
Current production exceeds capacity - reduce by: 1,048,935 kg/hr (50.0%)


### 17.2 Multi-Variable Optimization (Flow Rate + Split Factors)

Using Nelder-Mead simplex algorithm to optimize both inlet flow rate and splitter distribution simultaneously.

In [48]:
print("=" * 50)
print("=== MULTI-VARIABLE OPTIMIZATION ===")
print("Optimizing: Inlet Flow + Splitter Distribution")
print("=" * 50)

# Get the splitter that distributes flow to the 3 compressor trains
optimizer_splitter = process_system.getUnit("Test Splitter2")
base_flow = float(original_flow)

# Define manipulated variables using JPype BiConsumer interface
@JImplements("java.util.function.BiConsumer")
class FlowSetter:
    @JOverride
    def accept(self, proc, val):
        inlet = proc.getUnit("Inlet Stream")
        inlet.setFlowRate(float(val), "kg/hr")

@JImplements("java.util.function.BiConsumer")
class BalanceSetter:
    @JOverride
    def accept(self, proc, val):
        split_unit = proc.getUnit("Test Splitter2")
        base_val = 1.0 / 3.0
        f1 = max(0.2, min(0.5, base_val + float(val)))
        f3 = max(0.2, min(0.5, base_val - float(val)))
        f2 = 1.0 - f1 - f3
        split_unit.setSplitFactors([f1, f2, f3])

# Variable 1: Inlet flow rate
flow_var = ManipulatedVariable(
    "InletFlow",
    base_flow * 0.90,  # lower bound
    base_flow * 1.15,  # upper bound
    "kg/hr",
    FlowSetter()
)

# Variable 2: Balance factor (controls split distribution)
# -0.10 to +0.10, where 0 = equal split
balance_var = ManipulatedVariable(
    "BalanceFactor",
    -0.10,  # lower bound
    0.10,   # upper bound
    "fraction",
    BalanceSetter()
)

multi_variables = Arrays.asList(flow_var, balance_var)

# Configure multi-variable optimization
multi_var_config = OptimizationConfig(base_flow * 0.90, base_flow * 1.15) \
    .rateUnit("kg/hr") \
    .tolerance(base_flow * 0.005) \
    .maxIterations(30) \
    .defaultUtilizationLimit(1.0) \
    .searchMode(SearchMode.NELDER_MEAD_SCORE)

print("\nRunning Nelder-Mead multi-variable optimization...")

# Run multi-variable optimization
multi_var_result = optimizer.optimize(
    process_system, 
    multi_variables,
    multi_var_config, 
    Collections.singletonList(throughput_objective), 
    Collections.emptyList()
)

# Get optimized values
decision_vars = multi_var_result.getDecisionVariables()
best_flow = decision_vars.getOrDefault("InletFlow", JDouble.valueOf(base_flow))
best_balance_factor = decision_vars.getOrDefault("BalanceFactor", JDouble.valueOf(0.0))

# Calculate final split factors
base = 1.0 / 3.0
best_f1 = max(0.2, min(0.5, base + float(best_balance_factor)))
best_f3 = max(0.2, min(0.5, base - float(best_balance_factor)))
best_f2 = 1.0 - best_f1 - best_f3

print("\n" + "=" * 50)
print("=== MULTI-VARIABLE OPTIMIZATION RESULTS ===")
print("=" * 50)
print(f"Optimal inlet flow: {float(best_flow):,.0f} kg/hr")
print(f"Optimal balance factor: {float(best_balance_factor):.3f}")
print(f"Optimal split factors:")
print(f"  Train 1 (ups1): {best_f1 * 100:.1f}%")
print(f"  Train 2 (ups2): {best_f2 * 100:.1f}%")
print(f"  Train 3 (ups3): {best_f3 * 100:.1f}%")
print(f"Feasible: {multi_var_result.isFeasible()}")
print(f"Iterations: {multi_var_result.getIterations()}")

if multi_var_result.getBottleneck() is not None:
    print(f"Limiting equipment: {multi_var_result.getBottleneck().getName()}")
    print(f"Max utilization: {multi_var_result.getBottleneckUtilization() * 100.0:.1f}%")

=== MULTI-VARIABLE OPTIMIZATION ===
Optimizing: Inlet Flow + Splitter Distribution

Running Nelder-Mead multi-variable optimization...

=== MULTI-VARIABLE OPTIMIZATION RESULTS ===
Optimal inlet flow: 2,412,551 kg/hr
Optimal balance factor: -0.100
Optimal split factors:
  Train 1 (ups1): 23.3%
  Train 2 (ups2): 33.3%
  Train 3 (ups3): 43.3%
Feasible: False
Iterations: 30
Limiting equipment: ups2 Compressor
Max utilization: 104010.1%


In [49]:
# Compare with initial flow
improvement = (float(best_flow) - float(original_flow)) / float(original_flow) * 100

print("=" * 50)
print("=== COMPARISON ===")
print("=" * 50)
print(f"Initial flow: {float(original_flow):,.0f} kg/hr")
print(f"Optimized flow: {float(best_flow):,.0f} kg/hr")
print(f"Improvement: {improvement:.1f}%")

=== COMPARISON ===
Initial flow: 2,097,871 kg/hr
Optimized flow: 2,412,551 kg/hr
Improvement: 15.0%


### 17.3 Pareto Multi-Objective Optimization

Demonstrate Pareto optimization with two objectives:
1. **Maximize throughput** - Production rate
2. **Minimize total compressor power** - Energy consumption

This generates a Pareto front showing the trade-offs between throughput and energy efficiency.

In [50]:
print("=" * 50)
print("=== PARETO MULTI-OBJECTIVE OPTIMIZATION ===")
print("Objectives: Maximize Throughput vs Minimize Power")
print("=" * 50)

# Define power minimization objective with error handling
@JImplements("java.util.function.ToDoubleFunction")
class PowerEvaluator:
    @JOverride
    def applyAsDouble(self, proc):
        try:
            total_power = 0.0
            for eq in proc.getUnitOperations():
                eq_name = str(eq.getName()) if eq.getName() is not None else ""
                # Check if it's a Compressor and starts with "ups"
                class_name = str(eq.getClass().getSimpleName())
                if "Compressor" in class_name and eq_name.startswith("ups"):
                    power = eq.getPower("kW")
                    # Only add positive power values
                    if power > 0:
                        total_power += power
            return total_power
        except Exception:
            return float('nan')

power_objective = OptimizationObjective(
    "totalPower", 
    PowerEvaluator(), 
    1.0, 
    ObjectiveType.MINIMIZE
)

pareto_objectives = Arrays.asList(throughput_objective, power_objective)

# Configure Pareto optimization with reduced search range
pareto_config = OptimizationConfig(base_flow * 0.90, base_flow * 1.05) \
    .rateUnit("kg/hr") \
    .tolerance(base_flow * 0.01) \
    .maxIterations(10) \
    .defaultUtilizationLimit(1.0) \
    .searchMode(SearchMode.GOLDEN_SECTION_SCORE) \
    .paretoGridSize(5)  # 5 weight combinations

print("\nRunning Pareto optimization...")

# Run Pareto optimization
pareto_result = optimizer.optimizePareto(
    process_system, 
    inlet_stream, 
    pareto_config,
    pareto_objectives, 
    Collections.emptyList()
)

# Print Pareto results
print(f"\nPareto Front Size: {pareto_result.getParetoFrontSize()}")
print(f"Total Iterations: {pareto_result.getTotalIterations()}")
print("\nPareto Front:")
print(pareto_result.toMarkdownTable())

=== PARETO MULTI-OBJECTIVE OPTIMIZATION ===
Objectives: Maximize Throughput vs Minimize Power

Running Pareto optimization...

Pareto Front Size: 2
Total Iterations: 30

Pareto Front:
| # | Feasible | throughput | totalPower | Weights |
|---|---|---|---|---|
| 1 | no | 2174389.3812 | 76063.7759 | [0.75, 0.25] |
| 2 | no | 1894781.8899 | 66272.2190 | [0.00, 1.00] |



In [51]:
# Print utopia and nadir points
print("=" * 50)
print("Utopia Point (best individual values):")
for entry in pareto_result.getUtopiaPoint().entrySet():
    print(f"  {entry.getKey()}: {entry.getValue():.2f}")

print("\nNadir Point (worst values on Pareto front):")
for entry in pareto_result.getNadirPoint().entrySet():
    print(f"  {entry.getKey()}: {entry.getValue():.2f}")

Utopia Point (best individual values):
  throughput: 2174389.38
  totalPower: 66272.22

Nadir Point (worst values on Pareto front):
  throughput: 1894781.89
  totalPower: 76063.78


In [52]:
# Restore best solution from single-variable optimization and apply optimized split factors
inlet_stream.setFlowRate(single_var_result.getOptimalRate(), "kg/hr")
optimizer_splitter.setSplitFactors([best_f1, best_f2, best_f3])
process_system.run()

# Show final compressor power utilization
print("=" * 50)
print("=== FINAL COMPRESSOR POWER UTILIZATION ===")
print("=" * 50)
for equip in process_system.getUnitOperations():
    if isinstance(equip, Compressor):
        c = equip
        name = str(c.getName())
        if name.startswith("ups"):
            power_util = 0.0
            if c.getDriver() is not None:
                max_pwr = c.getDriver().getMaxAvailablePowerAtSpeed(c.getSpeed())
                if max_pwr > 0:
                    power_util = c.getPower("kW") / max_pwr * 100
            print(f"  {name:15s}: Power={c.getPower('kW'):,.0f} kW, Utilization={power_util:.1f}%")

=== FINAL COMPRESSOR POWER UTILIZATION ===
  ups1 Compressor: Power=16,600 kW, Utilization=42.6%
  ups2 Compressor: Power=22,982 kW, Utilization=55.1%
  ups3 Compressor: Power=27,758 kW, Utilization=64.1%


## Summary

This notebook demonstrated:
1. Building a complex multi-train gas processing system with NeqSim
2. Configuring compressors with performance charts and gas turbine drivers
3. Analyzing pipe velocities against design limits
4. Monitoring equipment capacity utilization
5. Detecting system bottlenecks and constraints
6. **Production optimization using `ProductionOptimizer`:**
   - Single-variable optimization (flow rate) with Golden Section search
   - Multi-variable optimization (flow + split factors) with Nelder-Mead
   - Pareto multi-objective optimization (throughput vs power)

The simulation includes 4 parallel processing trains feeding into 3 compressor trains, with detailed analysis of compressor operating margins including surge, stonewall, speed limits, and power consumption.